In [1]:
import torch
from transformers import AdamW, AutoTokenizer, AutoModelForSequenceClassification, BertTokenizer, BertConfig, BertForSequenceClassification, BertPreTrainedModel, BertModel
import json
from torch.utils.data import TensorDataset, random_split, Subset
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import TrainingArguments, Trainer
from torch import nn
import torch.nn.functional as F
from transformers import get_linear_schedule_with_warmup
from sklearn.metrics import f1_score, precision_recall_fscore_support
import pickle

In [8]:
device = 'cuda'

In [23]:
test = json.load(open('.\document retrieval_data\claim_test_BM25F_V3.json', 'r', encoding='utf-8'))

In [9]:
test = json.load(open('.\data\CHEF_test_hfl_pretraineds_0511sentBase_document_article_epoch50_0519_BM25F_V3_new.json', 'r', encoding='utf-8'))
labels_ = [row['label'] for row in test]

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 test = json.load(open('..\data\CHEF_test_hfl_pretraineds_0511sentBase_document_article_e     │
│   2 labels_ = [row['label'] for row in test]                                                     │
│   3                                                                                              │
│                                                                                                  │
│ c:\Users\NUI\anaconda3\envs\pytorch-gpu\lib\site-packages\IPython\core\interactiveshell.py:282   │
│ in _modified_open                                                                                │
│                                                                                                  │
│    279 │   │   │   "you can use builtins' open."                                                 │
│    280 │   │   )                                                                                 │
│    281 │                                                                                         │
│ ❱  282 │   return io_open(file, *args, **kwargs)                                                 │
│    283                                                                                           │
│    284 class InteractiveShell(SingletonConfigurable):                                            │
│    285 │   """An enhanced, interactive shell for Python."""                                      │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
FileNotFoundError: [Errno 2] No such file or directory: 
'..\\data\\CHEF_test_hfl_pretraineds_0511sentBase_document_article_epoch50_0519_BM25F_V3_new.json'

In [10]:
model_state_dict = torch.load('19_BM25F_V3_9297_epoch__11\\pytorch_model.bin', map_location=torch.device('cuda'))
model = BertForSequenceClassification.from_pretrained('D:\\Vic\\GitHub\\Team3068_FactualTextRetrieval\\sbert\\hfl_pretraineds_0511sentBase_document_article_epoch50_0519', state_dict=model_state_dict, num_labels=3,output_attentions = False,output_hidden_states = False)
model.to(device)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 model_state_dict = torch.load('19_BM25F_V3_9297_epoch__11\\pytorch_model.bin', map_locat     │
│   2 model = BertForSequenceClassification.from_pretrained('D:\\Vic\\GitHub\\Team3068_Factual     │
│   3 model.to(device)                                                                             │
│   4                                                                                              │
│                                                                                                  │
│ c:\Users\NUI\anaconda3\envs\pytorch-gpu\lib\site-packages\torch\serialization.py:791 in load     │
│                                                                                                  │
│    788 │   if 'encoding' not in pickle_load_args.keys():                                         │
│    789 │   │   pickle_load_args['encoding'] = 'utf-8'                                            │
│    790 │                                                                                         │
│ ❱  791 │   with _open_file_like(f, 'rb') as opened_file:                                         │
│    792 │   │   if _is_zipfile(opened_file):                                                      │
│    793 │   │   │   # The zipfile reader is going to advance the current file position.           │
│    794 │   │   │   # If we want to actually tail call to torch.jit.load, we need to              │
│                                                                                                  │
│ c:\Users\NUI\anaconda3\envs\pytorch-gpu\lib\site-packages\torch\serialization.py:271 in          │
│ _open_file_like                                                                                  │
│                                                                                                  │
│    268                                                                                           │
│    269 def _open_file_like(name_or_buffer, mode):                                                │
│    270 │   if _is_path(name_or_buffer):                                                          │
│ ❱  271 │   │   return _open_file(name_or_buffer, mode)                                           │
│    272 │   else:                                                                                 │
│    273 │   │   if 'w' in mode:                                                                   │
│    274 │   │   │   return _open_buffer_writer(name_or_buffer)                                    │
│                                                                                                  │
│ c:\Users\NUI\anaconda3\envs\pytorch-gpu\lib\site-packages\torch\serialization.py:252 in __init__ │
│                                                                                                  │
│    249                                                                                           │
│    250 class _open_file(_opener):                                                                │
│    251 │   def __init__(self, name, mode):                                                       │
│ ❱  252 │   │   super().__init__(open(name, mode))                                                │
│    253 │                                                                                         │
│    254 │   def __exit__(self, *args):                                                            │
│    255 │   │   self.file_like.close()                                                            │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
FileNotFoundError: [Errno 2] No such file or directory: '19_B

In [11]:
tokenizer = BertTokenizer.from_pretrained('D:\\hfl_pretraineds_0511sentBase_document_article_epoch50_0519')

In [12]:
sentence = json.load(open('D:\\download\\比賽\\CHEF-sbert\\Pipeline\\Data\\claim_cossim_hfl_pretraineds_0511sentBase_document_article_epoch50_0519_BM25F_V3.json', 'r', encoding='utf-8'))
sentence = sentence[11620:]

In [13]:
input_ids = []
attention_masks = []
labels = []
for i in range(len(sentence)):
    encoded_dict = tokenizer.encode_plus(
        sentence[i],  # Sentence to encode.
        add_special_tokens=False,  # Add '[CLS]' and '[SEP]'
        max_length= 512,  # Pad & truncate all sentences.
        padding='max_length',
        return_attention_mask=True,  # Construct attn. masks.
        return_tensors='pt',  # Return pytorch tensors.
        truncation=True
    )
    # Add the encoded sentence to the list.
    input_ids.append(encoded_dict['input_ids'])
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])
    #labels.append(2)
    labels.append(labels_[i])

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:19                                                                                   │
│                                                                                                  │
│   16 │   # And its attention mask (simply differentiates padding from non-padding).              │
│   17 │   attention_masks.append(encoded_dict['attention_mask'])                                  │
│   18 │   #labels.append(2)                                                                       │
│ ❱ 19 │   labels.append(labels_[i])                                                               │
│   20                                                                                             │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'labels_' is not defined

In [14]:
input_ids = torch.cat(input_ids, dim=0).to(device)
attention_masks = torch.cat(attention_masks, dim=0).to(device)
labels = torch.tensor(labels, device='cuda')
test_dataset = TensorDataset(input_ids, attention_masks,labels)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:4                                                                                    │
│                                                                                                  │
│   1 input_ids = torch.cat(input_ids, dim=0).to(device)                                           │
│   2 attention_masks = torch.cat(attention_masks, dim=0).to(device)                               │
│   3 labels = torch.tensor(labels, device='cuda')                                                 │
│ ❱ 4 test_dataset = TensorDataset(input_ids, attention_masks,labels)                              │
│   5                                                                                              │
│                                                                                                  │
│ c:\Users\NUI\anaconda3\envs\pytorch-gpu\lib\site-packages\torch\utils\data\dataset.py:192 in     │
│ __init__                                                                                         │
│                                                                                                  │
│   189 │   tensors: Tuple[Tensor, ...]                                                            │
│   190 │                                                                                          │
│   191 │   def __init__(self, *tensors: Tensor) -> None:                                          │
│ ❱ 192 │   │   assert all(tensors[0].size(0) == tensor.size(0) for tensor in tensors), "Size mi   │
│   193 │   │   self.tensors = tensors                                                             │
│   194 │                                                                                          │
│   195 │   def __getitem__(self, index):                                                          │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
AssertionError: Size mismatch between tensors

In [15]:
test_dataloader = DataLoader(
        test_dataset,
        sampler=SequentialSampler(test_dataset),   
        batch_size=8 
    )

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│   1 test_dataloader = DataLoader(                                                                │
│ ❱ 2 │   │   test_dataset,                                                                        │
│   3 │   │   sampler=SequentialSampler(test_dataset),                                             │
│   4 │   │   batch_size=8                                                                         │
│   5 │   )                                                                                        │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'test_dataset' is not defined

In [16]:
import time
import numpy as np
import os
from collections import Counter

import datetime
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [17]:
def format_time(elapsed):
    elapsed_rounded = int(round((elapsed)))
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [18]:
best_microf1 = 0
best_macrof1 = 0
best_recall = 0
best_precision = 0
best_prediction = None
best_ground_truth = None
t0 = time.time()
# Put the model in evaluation mode
#model.forward()
model.eval()
print("Running Validation...")
# Tracking variables
total_eval_accuracy = 0
total_eval_loss = 0
nb_eval_steps = 0
all_prediction = np.array([])
all_ground_truth = np.array([])
all_logits = np.array([])
# Evaluate data for one epoch
for batch in test_dataloader:
    # Unpack this training batch from our dataloader.
    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)
    b_labels = batch[2].to(device)
    with torch.no_grad():
        outputs = model(
            b_input_ids, 
            token_type_ids=None, 
            attention_mask=b_input_mask,
            labels=b_labels
        )
        loss, logits = outputs[0], outputs[1]
    # Accumulate the validation loss.
    total_eval_loss += loss.sum().item()
    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    pred_flat = np.argmax(logits, axis=1).flatten()
    labels_flat = label_ids.flatten()
    all_prediction = np.concatenate((all_prediction, pred_flat), axis=None)
    all_ground_truth = np.concatenate((all_ground_truth, labels_flat), axis=None)
    if len(all_logits) == 0:
        all_logits = logits
    else:
        all_logits = np.concatenate((all_logits, logits), axis=0)

# Calculate the average loss over all of the batches.
avg_val_loss = total_eval_loss / len(test_dataloader)
# Measure how long the validation run took.
validation_time = format_time(time.time() - t0)
print('Validation Elapsed: {:}.'.format(validation_time))
c = Counter()
for pred in all_prediction:
    c[int(pred)] += 1
print(c)
pre, recall, f1, _ = precision_recall_fscore_support(all_ground_truth, all_prediction, average='micro')
print("       F1 (micro): {:.2%}".format(f1))
microf1 = f1
pre, recall, f1, _ = precision_recall_fscore_support(all_ground_truth, all_prediction, average='macro')
print("Precision (macro): {:.2%}".format(pre))
print("   Recall (macro): {:.2%}".format(recall))
print("       F1 (macro): {:.2%}".format(f1))

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:10                                                                                   │
│                                                                                                  │
│    7 t0 = time.time()                                                                            │
│    8 # Put the model in evaluation mode                                                          │
│    9 #model.forward()                                                                            │
│ ❱ 10 model.eval()                                                                                │
│   11 print("Running Validation...")                                                              │
│   12 # Tracking variables                                                                        │
│   13 total_eval_accuracy = 0                                                                     │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'model' is not defined

In [19]:
with open('D:\\download\\比賽\\all_prediction_predict_hfl_pretraineds_0511sentBase_document_article_epoch50_0519_BM25F_V3_9297.pickle', 'wb') as f:
    pickle.dump(all_prediction, f)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│   1 with open('D:\\download\\比賽\\all_prediction_predict_hfl_pretraineds_0511sentBase_docum     │
│ ❱ 2 │   pickle.dump(all_prediction, f)                                                           │
│   3                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'all_prediction' is not defined

In [20]:
import pickle
with open('D:\\download\\比賽\\CHEF-sbert\\Pipeline\\Data\\all_prediction_hfl_pretraineds_0511sentBase_document_article_epoch50_0519_BM25F_V3_9297_epoch_11.pickle', 'rb') as f:
    label = (pickle.load(f))

In [21]:
all_prediction

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 all_prediction                                                                               │
│   2                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'all_prediction' is not defined